In [1]:
import os
from typing import ClassVar

import psycopg2
from dotenv import load_dotenv
from smolagents import LiteLLMModel, Tool, ToolCallingAgent


load_dotenv()

True

In [2]:
db_params = {
    "host": "127.0.0.1",
    "port": 5432,
    "database": os.environ["POSTGRES_DB"],
    "user": os.environ["POSTGRES_USER"],
    "password": os.environ["POSTGRES_PASSWORD"],
}


def get_available_tables(schema: str = "public") -> list[str]:
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    cursor.execute(
        """
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = %s
        AND table_type = 'BASE TABLE';
        """,
        (schema,),
    )
    tables = [row[0] for row in cursor.fetchall()]
    conn.close()
    if not tables:
        return f"No tables found in schema '{schema}'."
    return f"Available tables:\n - " + "\n - ".join(tables)


def get_table_schema(table_name: str = "resumes") -> str:
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    cursor.execute(
        """
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = %s;
        """,
        (table_name,),
    )
    columns = cursor.fetchall()
    conn.close()

    schema_lines = [f" - {col[0]} ({col[1]})" for col in columns]
    return f"Table Schema for '{table_name}':\n" + "\n".join(schema_lines)


def get_table_preview(table_name: str = "resumes", limit: int = 5) -> str:
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    cursor.execute(f"SELECT * FROM {table_name} LIMIT %s;", (limit,))
    rows = cursor.fetchall()
    colnames = [desc[0] for desc in cursor.description]
    conn.close()

    header = " | ".join(colnames)
    lines = [" | ".join(str(cell) for cell in row) for row in rows]

    return f"First {limit} rows from '{table_name}':\n" + header + "\n" + "\n".join(lines)

print(get_available_tables())
print(get_table_schema())
print(get_table_preview())

Available tables:
 - resumes
Table Schema for 'resumes':
 - id (integer)
 - contact_info (jsonb)
 - experience (jsonb)
 - education (jsonb)
 - portfolio (jsonb)
 - languages (ARRAY)
 - skills (ARRAY)
 - certifications (ARRAY)
 - hobbies (ARRAY)
 - name (text)
 - gender (text)
 - title (text)
 - summary (text)
First 5 rows from 'resumes':
id | name | gender | title | summary | contact_info | skills | experience | education | languages | certifications | hobbies | portfolio
1 | Зыкова Валерия Кузьминична | женский | Mobile Developer | Опытный мобильный разработчик с 5-летним стажем в разработке и оптимизации мобильных приложений. Обладаю глубоким пониманием современных технологий и платформ, таких как iOS и Android. Сильные навыки в проектировании, разработке и тестировании приложений, а также в работе в команде и управлении проектами. Ищу возможность применить свои навыки и знания в динамичной и инновационной компании. | {'email': 'valeriya.zykova@example.com', 'phone': '+7 (808) 262-35

In [12]:
from smolagents import CodeAgent, OpenAIServerModel, tool


class SQLExecutorTool(Tool):
    """A tool that executes SQL queries on a PostgreSQL database.

    Args:
        query (str): A SQL query to execute.

    Returns:
        string: Query results as a string.

    """

    name = "sql_executor"
    description = "Executes SQL queries on the PostgreSQL database. Returns query results as a string."

    inputs: ClassVar[dict[str, dict[str, str]]] = {
        "query": {"type": "string", "description": "A valid SQL query string of postgres dialect"}
    }

    output_type = "string"

    def __init__(self, db_params: dict) -> None:
        self.db_params = db_params
        self.is_initialized = True

    def forward(self, query: str):  # noqa: ANN201
        try:
            conn = psycopg2.connect(
                host=self.db_params["host"],
                port=self.db_params["port"],
                database=self.db_params["database"],
                user=self.db_params["user"],
                password=self.db_params["password"],
            )
            cursor = conn.cursor()
            cursor.execute(query)

            try:
                results = cursor.fetchall()
            except psycopg2.ProgrammingError:
                results = "Query executed successfully, but no results to fetch."

            conn.commit()

        except Exception as e:
            results = f"SQL Execution Error: {e}"
        finally:
            if conn:
                conn.close()
        return str(results)

sql_tool = SQLExecutorTool(db_params)


model = OpenAIServerModel(
    model_id="llama3.2:1b",
    api_base="http://localhost:11434/v1",  # Leave this blank to query OpenAI servers.
    api_key="ollama",
)

agent = ToolCallingAgent(
    tools=[sql_tool], model=model
)

prompt = f"выведи имена devops инженеров? {get_available_tables(), get_table_schema(), get_table_preview()}"

final_output = agent.run(f"{prompt}")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ выведи имена devops инженеров? ('Available tables:\n - resumes', "Table Schema for 'resumes':\n - id            │
│ (integer)\n - contact_info (jsonb)\n - experience (jsonb)\n - education (jsonb)\n - portfolio (jsonb)\n -       │
│ languages (ARRAY)\n - skills (ARRAY)\n - certifications (ARRAY)\n - hobbies (ARRAY)\n - name (text)\n - gender  │
│ (text)\n - title (text)\n - summary (text)", "First 5 rows from 'resumes':\nid | name | gender | title |        │
│ summary | contact_info | skills | experience | education | languages | certifications | hobbies | portfolio\n1  │
│ | Зыкова Валерия Кузьминична | женский | Mobile Developer | Опытный мобильный разработчик с 5-летним стажем в   │
│ разработке и оптимизации мобильных приложений. Обладаю глубоким пониманием современных технологий и платформ,   │
│ таких как iOS и Android. Сильные навыки в проектировании, разработке и тестировании приложений, а также в       │
│ работе в команде и управлении проектами. Ищу возможность применить свои навыки и знания в динамичной и          │
│ инновационной компании. | {'email': 'valeriya.zykova@example.com', 'phone': '+7 (808) 262-35-84', 'github':     │
│ 'github.com/valeriya-zykova', 'linkedin': 'linkedin.com/in/valeriya-zykova', 'location': 'Чехия'} | ['Swift',   │
│ 'Kotlin', 'React Native', 'Firebase', 'Git', 'Agile/Scrum', 'UX/UI Design', 'RESTful APIs', 'CI/CD',            │
│ 'Test-Driven Development'\] | [{'company': 'TechSolutions', 'end_date': '2023-05-31', 'job_title': 'Mobile      │
│ Developer', 'start_date': '2018-06-01', 'achievements': ['Разработала и запустила 3 мобильных приложения для    │
│ iOS и Android, которые достигли более 100 000 загрузок.', 'Оптимизировала производительность приложений,        │
│ сократив время загрузки на 30%.', 'Внедрила CI/CD пайплайны, что сократило время развертывания на 50%.',        │
│ 'РаЬотала в мультидисциплинарной команде, координируя усилия разработчиков, дизайнеров и тестировщиков.'\]}] |  │
│ [{'degree': 'Бакалавр информационных технологий', 'details': 'Специализация: Программная инженерия',            │
│ 'end_date': '2017-06-30', 'start_date': '2013-09-01', 'institution': 'Санкт-Петербургский Политехнический       │
│ Университет'}\] | ['Русский – родной', 'Английский – B2', 'Чешский – A2'\] | ['Google Certified Professional    │
│ Cloud Developer', 'Apple Developer Program'\] | ['Фотография', 'Путешествия', 'Программирование в свободное     │
│ время'\] | [{'link': 'github.com/valeriya-zykova/TravelApp', 'name': 'TravelApp', 'description': 'Мобильное     │
│ приложение для планирования путешествий. Использовались технологии: Swift, Firebase, MapKit. Приложение         │
│ позволяет пользователям создавать маршруты, добавлять точки интереса и делиться планами с друзьями.'}, {'link': │
│ 'github.com/valeriya-zykova/FitnessTracker', 'name': 'FitnessTracker', 'description': 'Приложение для           │
│ отслеживания физической активности. Использовались технологии: Kotlin, Google Fit API, Room Database.           │
│ Приложение позволяет пользователям отслеживать свои тренировки, устанавливать цели и получать уведомления о     │
│ прогрессе.'}\]\n2 | Фадеева Татьяна Кузьминична | женский | Product Owner | Опытный Product Owner с 9-летним    │
│ стажем в управлении продуктами и командами. Сильные навыки в аджайл-методологиях, стратегическом планировании и │
│ взаимодействии с ключевыми заинтересованными сторонами. Успешно запустила и масштабировала несколько продуктов, │
│ достигнув значительных бизнес-результатов и улучшив пользовательский опыт. | {'email':                          │
│ 'tatyana.fadeeva@example.com', 'phone': '8 764 180 2753', 'github': 'github.com/tatyana-fadeeva', 'linkedin':   │
│ 'linkedin.com/in/tatyana-fadeeva', 'location': 'Мали'}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error while generating or parsing output:
The JSON blob you used is invalid

[Step 1: Duration 10.11 seconds| Input tokens: 2,048 | Output tokens: 303]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

[Step 3: Duration 0.54 seconds| Input tokens: 4,417 | Output tokens: 514]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'There is no orders relation in the database'}          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final answer: There is no orders relation in the database

[Step 4: Duration 0.62 seconds| Input tokens: 5,759 | Output tokens: 546]

In [5]:
from smolagents import CodeAgent, OpenAIServerModel, tool


@tool
def sql_engine(query: str) -> str:
    """Allow perform SQL queries on the table. Returns a string representation of the result.

    The table is named 'resumes'

    Table Schema for 'resumes':
     - id (integer)
     - contact_info (jsonb)
     - experience (jsonb)
     - education (jsonb)
     - portfolio (jsonb)
     - languages (ARRAY)
     - skills (ARRAY)
     - certifications (ARRAY)
     - hobbies (ARRAY)
     - name (text)
     - gender (text)
     - title (text)
     - summary (text)

    First 5 rows from resumestable preview:
    id | name | gender | title | summary | contact_info | skills | experience | education | languages | certifications | hobbies | portfolio
    1 | Зыкова Валерия Кузьминична | женский | Mobile Developer | Опытный мобильный разработчик с 5-летним стажем в разработке и оптимизации мобильных приложений. Обладаю глубоким пониманием современных технологий и платформ, таких как iOS и Android. Сильные навыки в проектировании, разработке и тестировании приложений, а также в работе в команде и управлении проектами. Ищу возможность применить свои навыки и знания в динамичной и инновационной компании. | {'email': 'valeriya.zykova@example.com', 'phone': '+7 (808) 262-35-84', 'github': 'github.com/valeriya-zykova', 'linkedin': 'linkedin.com/in/valeriya-zykova', 'location': 'Чехия'} | ['Swift', 'Kotlin', 'React Native', 'Firebase', 'Git', 'Agile/Scrum', 'UX/UI Design', 'RESTful APIs', 'CI/CD', 'Test-Driven Development'] | [{'company': 'TechSolutions', 'end_date': '2023-05-31', 'job_title': 'Mobile Developer', 'start_date': '2018-06-01', 'achievements': ['Разработала и запустила 3 мобильных приложения для iOS и Android, которые достигли более 100 000 загрузок.', 'Оптимизировала производительность приложений, сократив время загрузки на 30%.', 'Внедрила CI/CD пайплайны, что сократило время развертывания на 50%.', 'РаЬотала в мультидисциплинарной команде, координируя усилия разработчиков, дизайнеров и тестировщиков.']}] | [{'degree': 'Бакалавр информационных технологий', 'details': 'Специализация: Программная инженерия', 'end_date': '2017-06-30', 'start_date': '2013-09-01', 'institution': 'Санкт-Петербургский Политехнический Университет'}] | ['Русский – родной', 'Английский – B2', 'Чешский – A2'] | ['Google Certified Professional Cloud Developer', 'Apple Developer Program'] | ['Фотография', 'Путешествия', 'Программирование в свободное время'] | [{'link': 'github.com/valeriya-zykova/TravelApp', 'name': 'TravelApp', 'description': 'Мобильное приложение для планирования путешествий. Использовались технологии: Swift, Firebase, MapKit. Приложение позволяет пользователям создавать маршруты, добавлять точки интереса и делиться планами с друзьями.'}, {'link': 'github.com/valeriya-zykova/FitnessTracker', 'name': 'FitnessTracker', 'description': 'Приложение для отслеживания физической активности. Использовались технологии: Kotlin, Google Fit API, Room Database. Приложение позволяет пользователям отслеживать свои тренировки, устанавливать цели и получать уведомления о прогрессе.'}]
    2 | Фадеева Татьяна Кузьминична | женский | Product Owner | Опытный Product Owner с 9-летним стажем в управлении продуктами и командами. Сильные навыки в аджайл-методологиях, стратегическом планировании и взаимодействии с ключевыми заинтересованными сторонами. Успешно запустила и масштабировала несколько продуктов, достигнув значительных бизнес-результатов и улучшив пользовательский опыт. | {'email': 'tatyana.fadeeva@example.com', 'phone': '8 764 180 2753', 'github': 'github.com/tatyana-fadeeva', 'linkedin': 'linkedin.com/in/tatyana-fadeeva', 'location': 'Мали'} | ['Agile и Scrum', 'Product Roadmap', 'User Story Mapping', 'Stakeholder Management', 'Data-Driven Decision Making', 'Leadership', 'Team Building', 'UX/UI Design Basics', 'Jira', 'Confluence'] | [{'company': 'Tech Solutions', 'end_date': '2023', 'job_title': 'Product Owner', 'start_date': '2018', 'achievements': ['Успешно запустила и масштабировала продукт для автоматизации бизнес-процессов, что привело к увеличению эффективности на 30% и росту выручки на 25%.', 'Разработала и внедрила стратегию улучшения пользовательского опыта, что повысило удовлетворенность клиентов на 20%.', 'Создала и поддерживала продукт-бэклог, обеспечивая четкое понимание приоритетов и целей команды.', 'Сотрудничала с ключевыми заинтересованными сторонами, включая руководство и технические команды, для выработки согласованных решений.']}, {'company': 'Innovatech', 'end_date': '2018', 'job_title': 'Product Manager', 'start_date': '2014', 'achievements': ['Управляла жизненным циклом продукта от идеи до запуска, обеспечивая своевременное выполнение всех этапов.', 'Разработала и внедрила методику оценки и анализа пользовательского поведения, что позволило оптимизировать функциональность продукта и повысить конверсию на 15%.', 'Организовала и проводила регулярные встречи с командой разработки, обеспечивая эффективное взаимодействие и выполнение спринтов.']}] | [{'degree': 'Магистр', 'details': 'Менеджмент', 'end_date': '2012', 'start_date': '2007', 'institution': 'Московский Государственный Университет'}] | ['Русский – родной', 'Английский – B2'] | ['Certified Scrum Product Owner (CSPO)', 'Google Analytics – Advanced'] | ['Путешествия', 'Фотография', 'Чтение книг по управлению продуктами'] | [{'link': 'github.com/tatyana-fadeeva/automation-project', 'name': 'Автоматизация бизнес-процессов', 'description': 'Разработка и запуск продукта для автоматизации бизнес-процессов, который повысил эффективность работы компании на 30% и увеличил выручку на 25%.'}, {'link': 'github.com/tatyana-fadeeva/ux-improvement', 'name': 'Улучшение пользовательского опыта', 'description': 'Проект по улучшению пользовательского опыта, который повысил удовлетворенность клиентов на 20% и увеличил конверсию на 15%.'}]
    3 | Горбачева Раиса Александровна | женский | Firmware Engineer | Опытный Firmware Engineer с 4 годами опыта в разработке и оптимизации встроенного программного обеспечения для различных устройств. Обладаю глубоким пониманием архитектуры микроконтроллеров и встраиваемых систем, а также сильными навыками в области C/C++ и Python. Имею успешный опыт работы в команде и управления проектами, что позволяет эффективно достигать поставленных целей. | {'email': 'raisa.gorbacheva@example.com', 'phone': '8 940 043 58 95', 'github': 'github.com/raisa-gorbacheva', 'linkedin': 'linkedin.com/in/raisa-gorbacheva', 'location': 'Индонезия'} | ['C/C++', 'Python', 'RTOS (FreeRTOS, Zephyr)', 'Microcontrollers (ARM, AVR)', 'Firmware Development', 'Debugging (JTAG, GDB)', 'Version Control (Git)', 'Agile Methodologies', 'Problem Solving', 'Team Collaboration'] | [{'company': 'Tech Innovations', 'end_date': '2023', 'job_title': 'Firmware Engineer', 'start_date': '2019', 'achievements': ['Разработала и внедрила firmware для IoT-устройств, что позволило увеличить производительность на 20% и снизить энергопотребление на 15%.', 'Успешно завершила проект по миграции legacy firmware на современные микроконтроллеры, сократив время разработки на 30%.', 'Создала и поддерживала документацию по разработке firmware, что улучшило процесс передачи знаний и сократило время наboarding новых сотрудников.']}] | [{'degree': 'Бакалавр', 'details': 'Специальность: Информатика и вычислительная техника', 'end_date': '2018', 'start_date': '2014', 'institution': 'Московский государственный университет'}] | ['Русский – родной', 'Английский – B2'] | ['Certified Firmware Developer (CFD) – 2021'] | ['Программирование микроконтроллеров', 'Путешествия', 'Фотография'] | [{'link': 'github.com/raisa-gorbacheva/iot-gateway-firmware', 'name': 'IoT Gateway Firmware', 'description': 'Разработка firmware для IoT-шлюза на базе ARM Cortex-M4. Использовались FreeRTOS, C/C++, и протоколы MQTT и CoAP. Проект позволил обеспечить надежную связь между устройствами и облачной платформой.'}, {'link': 'github.com/raisa-gorbacheva/low-power-sensor-node', 'name': 'Low-Power Sensor Node', 'description': 'Разработка firmware для низкоэнергетического датчика на базе AVR. Использовались оптимизации энергопотребления, такие как deep sleep режимы и энергоэффективные алгоритмы. Проект позволил увеличить время работы устройства от одной батареи на 50%.'}]
    4 | Маргарита Кирилловна Дорофеева | женский | DevOps Engineer | Опытный DevOps Engineer с 2 годами опыта в автоматизации инфраструктуры, оптимизации CI/CD пайплайнов и внедрении DevOps практик. Обладаю сильными навыками в работе с Kubernetes, Docker, и облачными платформами. Способна эффективно сотрудничать с командами разработки и операций, обеспечивая надежность и масштабируемость систем. | {'email': 'margarita.dorofeeva@example.com', 'phone': '8 (234) 422-6002', 'github': 'github.com/margarita-dorofeeva', 'linkedin': 'linkedin.com/in/margarita-dorofeeva', 'location': 'Эстония'} | ['Kubernetes', 'Docker', 'CI/CD (Jenkins, GitLab CI)', 'AWS', 'Terraform', 'Linux', 'Git', 'Python', 'Problem-solving', 'Teamwork'] | [{'company': 'TechSolutions', 'end_date': '2023', 'job_title': 'DevOps Engineer', 'start_date': '2021', 'achievements': ['Автоматизировала CI/CD пайплайны с использованием Jenkins, сократив время развертывания на 30%.', 'Внедрила мониторинг и логирование с помощью Prometheus и ELK Stack, улучшив видимость и управляемость инфраструктуры.', 'Настроила высокодоступную и масштабируемую инфраструктуру на AWS, снизив затраты на 20%.']}] | [{'degree': 'Бакалавр информационных технологий', 'details': 'Специализация: DevOps и облачные технологии', 'end_date': '2021', 'start_date': '2017', 'institution': 'Эстонский Технический Университет'}] | ['Русский – родной', 'Английский – B2'] | ['AWS Certified Solutions Architect – Associate', 'Google Cloud Professional DevOps Engineer'] | ['Программирование на Python', 'Путешествия', 'Фитнес'] | [{'link': 'github.com/margarita-dorofeeva/ci-cd-automation', 'name': 'Автоматизация CI/CD пайплайнов', 'description': 'Проект по автоматизации CI/CD пайплайнов с использованием Jenkins и GitLab CI. Включает скрипты для автоматического тестирования, сборки и развертывания приложений.'}, {'link': 'github.com/margarita-dorofeeva/monitoring-logging', 'name': 'Мониторинг и логирование', 'description': 'Проект по внедрению мониторинга и логирования с использованием Prometheus и ELK Stack. Включает конфигурации для сбора и анализа метрик и логов.'}]
    5 | Большакова Акулина Феликсовна | женский | Site Reliability Engineer | Опытный Site Reliability Engineer с 9-летним стажем в обеспечении высокой доступности и надежности систем. Специализируюсь на автоматизации процессов, мониторинге производительности и оптимизации инфраструктуры. Обладаю сильными навыками в области DevOps и системного администрирования, что позволяет эффективно решать сложные технические задачи и улучшать производительность систем. | {'email': 'akulina.bolshakova@example.com', 'phone': '+7 (710) 484-9177', 'github': 'github.com/akulina-bolshakova', 'linkedin': 'linkedin.com/in/akulina-bolshakova', 'location': 'Коста-Рика'} | ['DevOps', 'CI/CD', 'Kubernetes', 'Docker', 'Prometheus', 'Grafana', 'Terraform', 'AWS', 'Python', 'Linux', 'Problem-solving', 'Agile'] | [{'company': 'TechSolutions Inc.', 'end_date': '2023', 'job_title': 'Site Reliability Engineer', 'start_date': '2018', 'achievements': ['Разработала и внедрила CI/CD пайплайны, что сократило время развертывания на 40%.', 'Автоматизировала мониторинг и оповещения с использованием Prometheus и Grafana, повысив эффективность обнаружения и реагирования на инциденты на 30%.', 'Оптимизировала инфраструктуру с использованием Terraform, снизив затраты на облако на 25%.']}, {'company': 'WebTech Ltd.', 'end_date': '2018', 'job_title': 'System Administrator', 'start_date': '2014', 'achievements': ['Управление и поддержка серверной инфраструктуры на 100+ серверах, обеспечивающих бесперебойную работу критически важных систем.', 'Разработка и внедрение политик безопасности, что снизило количество инцидентов на 50%.', 'Внедрение контейнеризации с использованием Docker, улучшив масштабируемость и производительность приложений.']}] | [{'degree': 'Бакалавр информационных технологий', 'details': 'Специализация: Системное администрирование и сетевые технологии', 'end_date': '2013', 'start_date': '2009', 'institution': 'Национальный университет информационных технологий'}] | ['Русский – родной', 'Английский – C1'] | ['AWS Certified DevOps Engineer – Professional', 'Google Cloud Professional Cloud Architect'] | ['Путешествия', 'Программирование на Python', 'Фотография'] | [{'link': 'github.com/akulina-bolshakova/monitoring-automation', 'name': 'Автоматизация мониторинга и оповещений', 'description': 'Разработка и внедрение системы мониторинга с использованием Prometheus и Grafana для улучшения обнаружения и реагирования на инциденты.'}, {'link': 'github.com/akulina-bolshakova/terraform-optimization', 'name': 'Оптимизация инфраструктуры с использованием Terraform', 'description': 'Проект по оптимизации инфраструктуры с использованием Terraform, снизивший затраты на облако на 25%.'}]Use the sql_executor tool to run SQL queries only from resumes table.

    Args:
        query: The query to perform. This should be correct SQL.

    """
    output = ""
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    cursor.execute(query)

    try:
        output = cursor.fetchall()
    except psycopg2.ProgrammingError:
        output = "Query executed successfully, but no results to fetch."

    conn.commit()
    conn.close()
    return output

# model = LiteLLMModel(
#     model_id="ollama_chat/llama3.2:1b",
#     num_ctx=8192,
# )

model = OpenAIServerModel(
    model_id="llama3.2:1b",
    api_base="http://localhost:11434/v1",  # Leave this blank to query OpenAI servers.
    api_key="ollama",
)




agent = CodeAgent(
    tools=[sql_engine],
    model=model,
)

prompt = "выведи имена devops инженеров?"

final_output = agent.run(f"{prompt}")


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ выведи имена devops инженеров?                                                                                  │
│                                                                                                                 │
╰─ OpenAIServerModel - llama3.2:1b ───────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\n(.*?)\n``` was not found in it.
            Here is your code snippet:
            Давайте Try. Чтобы найти инженеры(devOps) по умолчанию, которые могут влиять на этот вопрос, I 
recommend utilizando базы данных или APIs, связанные с индустрией DevOps. Однако не найдя конкретных результатов 
для нашего конкретного вопроса, I can suggest algumas возможные методологии, что может помочь нам найти ответ:

1. **Сетевые API**: Мы можем использовать разные сеты API, которые предоставляют информации о людских 
resourceNameрах в различных индустриях, включая DevOps. Например, api.UMD.io, api.devsaa.com.
2. **Рассмотрение предназначения человека для определенного типа DevOps Ingeniera**: Мы можем попытаться найти 
профессиональный руководство о типах DevOps инженеров, которое peut предоставить конкретную информацию об этих 
индивидуальных инженерах.
3. **Рассмотрение различных источников и вариантов ответа**: Мы можем попредьстрировать различные资源ы, которые 
могут вносить влияние на решение вопроса о перечне DevOps инженеров.

Если вы хотели бы найти конкретные перечни DevOps инженеров по отдельным отраслям or региона. я приведу кнеже 
известных example профессией:

- **Дistributed DevOps**: *Dylan Brady*.  
- *Rakesh Agrawal* : 
 - *Pavel Pechtoria*
- *Kevin McKernan*: 
  **DevOps**

Кроме того, есть несколько онлайн-отратеный книжата и onlineresources с информацией о различных типах Devops 
инженеров

Действительно вы можете использовать его, чтобы получить более точную и полную информации.

Хочет ли вы продолжить или что еще quieres?
            Make sure to include code with the correct pattern, for instance:
            Thoughts: Your thoughts
            Code:
            ```py
            # Your python code here
            ```<end_code>
Make sure to provide correct code blobs.

[Step 1: Duration 5.79 seconds| Input tokens: 39 | Output tokens: 405]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

Code execution failed at line 'import requests' due to: InterpreterError: Import of requests is not allowed. 
Authorized imports are: ['statistics', 're', 'unicodedata', 'collections', 'stat', 'queue', 'random', 'math', 
'itertools', 'datetime', 'time'\]

[Step 3: Duration 6.51 seconds| Input tokens: 4,135 | Output tokens: 1,671]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[Step 4: Duration 6.26 seconds| Input tokens: 6,183 | Output tokens: 2,039]

KeyboardInterrupt: 